In [ ]:
# IMPORTS
%reload_ext autoreload
%autoreload 2
from typing import Any

import os
import sys
import defopt
import pickle
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from scipy.optimize import curve_fit
from suite2p.extraction import masks
import matplotlib
from matplotlib import cm

import cottage_analysis as cott
from cottage_analysis.depth_analysis.filepath.generate_filepaths import *
from cottage_analysis.imaging.common import find_frames
from cottage_analysis.imaging.common import imaging_loggers_formatting as format_loggers
from cottage_analysis.stimulus_structure import sphere_structure as vis_stim_structure
from cottage_analysis.depth_analysis.plotting.plotting_utils import *
from cottage_analysis.depth_analysis.depth_preprocess.process_params import *
from cottage_analysis.depth_analysis.depth_preprocess.process_trace import *

from cottage_analysis.io_module import harp

import flexiznam as flz
from pathlib import Path
from warnings import warn
from flexiznam.schema import Dataset

In [ ]:
project = 'hey2_3d-vision_foodres_20220101'
mouse = 'PZAH8.2f'
session = 'S20230213'
protocol = 'SpheresPermTubeReward'

# ----- SETUPS -----

choose_trials = 30
frame_rate = 15
speed_thr_cal = (
    0.2  # m/s, threshold for running speed when calculating depth neurons
)
is_calculate_dFF = True
###CHANGE THIS!!!###
if "Playback" in protocol:
    folder_no = 1
else:
    folder_no = 0

# ----- STEP1: Generate file path -----
print("---START STEP 1---", "\n", "Getting data filepaths...", flush=True)
flexilims_session = flz.get_flexilims_session(project_id=project)
all_protocol_recording_entries = get_all_recording_entries(project, mouse, session, protocol=protocol, flexilims_session=flexilims_session )
for irecording in range(len(all_protocol_recording_entries)):
    print(f'--Process recording {irecording}/{len(all_protocol_recording_entries)}', flush=True)

    (
        rawdata_folder,
        protocol_folder,
        analysis_folder,
        suite2p_folder,
        trace_folder,
    ) = generate_file_folders(
        project=project,
        mouse=mouse,
        session=session,
        protocol=protocol,
        all_protocol_recording_entries=all_protocol_recording_entries, 
        recording_no=irecording, 
        flexilims_session=flexilims_session
    )
    if not os.path.exists(analysis_folder/"plane0/"):
        os.makedirs(analysis_folder/"plane0/")

    if "Playback" in protocol:
        (
            rawdata_folder_closeloop,
            protocol_folder_closeloop,
            analysis_folder_closeloop,
            suite2p_folder_closeloop,
            trace_folder_closeloop,
        ) = generate_file_folders(
            project=project,
            mouse=mouse,
            session=session,
            protocol=protocol.replace("Playback", ""),
            all_protocol_recording_entries=all_protocol_recording_entries, 
            recording_no=irecording, 
            flexilims_session=flexilims_session
        )
        assert os.path.exists(analysis_folder_closeloop/"plane0/")
    print("---STEP 1 FINISHED.---", "\n", flush=True)

    # -----STEP2: Calculate dF/F from the raw fluorescence trace and Save -----
    print("---START STEP 2---", "\n", "Calculating dF/F for all ROIs...", flush=True)
    # Load suite2p files
    stat = np.load(suite2p_folder/"stat.npy", allow_pickle=True)
    ops = np.load(suite2p_folder/"ops.npy", allow_pickle=True)
    ops = ops.item()
    stats = np.load(suite2p_folder/"stat.npy", allow_pickle=True)
    iscell = np.load(suite2p_folder/"iscell.npy", allow_pickle=True)[:, 0]
    # F = np.load(trace_folder + 'F.npy', allow_pickle=True)
    Fast = np.load(trace_folder/"Fast.npy", allow_pickle=True)
    # Fneu = np.load(trace_folder + 'Fneu.npy', allow_pickle=True)
    # spks = np.load(trace_folder + 'spks.npy', allow_pickle=True)

    # The ROI no. for all cells (excluding non-cells)
    which_rois = (np.arange(Fast.shape[0]))[iscell.astype("bool")]

    # Calculate dF/F
    if is_calculate_dFF:
        dffs_ast = calculate_dFF(
            f=Fast, mode="gmm", n_components=2, verbose=True
        )
        np.save(trace_folder/"dffs_ast.npy", dffs_ast)
        print("dFFs saved.", flush=True)
        visualize_dffs = True
        if visualize_dffs:
            plt.figure(figsize=(10, 5))
            plt.plot(dffs_ast[0])
            plt.savefig(trace_folder/"dffs_ast_example.png")
    print("---STEP 2 FINISHED.---", "\n", flush=True)
    
    # -----STEP3: Align timestamps of visual stimulation loggers and save as img_VS.pickle file-----
    # This is trial-based analysis and has not included screen frame no.
    print("---START STEP 3---", "\n", "Align timestamps for all loggers...", flush=True)
    # photodiode_file = generate_filepaths.generate_logger_path(project=project,
    #                                                           mouse=mouse,
    #                                                           session=session,
    #                                                           protocol=protocol,
    #                                                           rawdata_root=rawdata_root,
    #                                                           root=root,
    #                                                           logger_name='PhotodiodeLog')

    # VS_photodiode_logger = format_loggers.format_VS_photodiode_logger(photodiode_file)
    # # plt.plot(VS_photodiode_logger['Photodiode'][100000:110000])
    # if not os.path.exists(trace_folder+'checker_plots/'):
    #     os.makedirs(trace_folder+'checker_plots/')
    # VS_photodiode_logger = find_frames.find_VS_frames(photodiode_df=VS_photodiode_logger, frame_rate=144, upper_thr=70,
    #                                                   lower_thr=30, plot=True, plot_start=100000, plot_range=2000,
    #                                                   plot_dir=trace_folder+'checker_plots/')
    # print('Photodiode signal extracted.', flush=True)

    # Load vis-stim param logger
    VS_param_file = generate_logger_path(
        project=project,
        mouse=mouse,
        session=session,
        protocol=protocol,
        logger_name="NewParams",    
        all_protocol_recording_entries=all_protocol_recording_entries, 
        recording_no=irecording, 
        flexilims_session=flexilims_session,
    )
    VS_param_logger = format_loggers.format_VS_param_logger(
        VS_param_file, which_protocol=protocol
    )

    # Load frame trigger logger and find frame triggers
    harpmessage_file = generate_logger_path(
        project=project,
        mouse=mouse,
        session=session,
        protocol=protocol,
        all_protocol_recording_entries=all_protocol_recording_entries, 
        recording_no=irecording, 
        flexilims_session=flexilims_session,
        logger_name="harp_message",
    )
    img_frame_logger = format_loggers.format_img_frame_logger(
        harpmessage_file, register_address=32
    )
    frame_number = ops["frames_per_folder"][folder_no]
    img_frame_logger = find_frames.find_imaging_frames(
        harp_message=img_frame_logger,
        frame_number=frame_number,
        exposure_time=0.0324 * 2,
        register_address=32,
        exposure_time_tolerance=0.001,
    )
    print("Frame trigger extracted.", flush=True)

    # Align vis-stim param timestamps
    # img_VS = align_timestamps.align_timestamps(VS_frames=VS_photodiode_logger, VS_params=VS_param_logger,
    #                                            imaging_frames=img_frame_logger)
    img_VS = pd.merge_asof(
        img_frame_logger,
        VS_param_logger,
        on="HarpTime",
        allow_exact_matches=True,
        direction="backward",
    )

    # Add running logger to the aligned dataframe (ALL DISTANCE IN METERS)
    rotary_encoder_file = generate_filepaths.generate_logger_path(
        project=project,
        mouse=mouse,
        session=session,
        protocol=protocol,
        rawdata_root=rawdata_root,
        root=root,
        logger_name="RotaryEncoder",
    )
    mousez_logger = pd.read_csv(
        rotary_encoder_file, sep=",", usecols=["HarpTime", "EyeZ", "MouseZ"]
    )
    img_VS = pd.merge_asof(
        img_VS,
        mousez_logger,
        on="HarpTime",
        allow_exact_matches=True,
        direction="backward",
    )

    img_VS.EyeZ = img_VS.EyeZ / 100  # Convert cm to m
    img_VS.MouseZ = img_VS.MouseZ / 100  # Convert cm to m
    img_VS.Depth = img_VS.Depth / 100  # Convert cm to m
    img_VS.Z0 = img_VS.Z0 / 100  # Convert cm to m

    depth_list = img_VS["Depth"].unique()
    depth_list = np.round(depth_list, 2)
    depth_list = depth_list[~np.isnan(depth_list)].tolist()
    depth_list.remove(-99.99)
    depth_list.sort()

    # print(img_VS[:20], flush=True)
    # Save img_VS
    with open(protocol_folder + "img_VS.pickle", "wb") as handle:
        pickle.dump(img_VS, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Timestamps aligned and saved.", flush=True)
    print("---STEP 3 FINISHED.---", "\n", flush=True)

In [ ]:
harpmessage_file = generate_logger_path(
    project=project,
    mouse=mouse,
    session=session,
    protocol=protocol,
    all_protocol_recording_entries=all_protocol_recording_entries, 
    recording_no=irecording, 
    flexilims_session=flexilims_session,
    logger_name="harp_message",
)

In [ ]:
msg = Path(str(harpmessage_file).replace('csv','bin'))
p_msg = trace_folder
p_msg = p_msg / (msg.stem + '.npz')
if p_msg.is_file():
    harp_messages = np.load(p_msg)
else:
    harp_messages = harp.load_harp(msg)
    p_msg.parent.mkdir(parents=True, exist_ok=True)
    np.savez(p_msg, **harp_messages)
